In [1]:
import tensorflow.keras.layers as layers
import tensorflow.keras as keras
import tensorflow as tf
import numpy as np
import utils
import dataProcessing
from models.model import AttentionResNet
from models.refModel import RefConvNet

print('Tensorflow version: {}'.format(tf.__version__))

Tensorflow version: 2.1.0


In [2]:
# TODO: Uncomment this, only for debuging
# @tf.function
def trainStep(model, x, y, loss_op, optimizer, train_loss, train_accuracy):
    with tf.GradientTape() as tape:
        predictions = model(x)
        loss = loss_op(y, predictions)

    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(y, predictions)
    
# TODO: Uncomment this, only for debuging
# @tf.function
def testStep(model, x, y, loss_op, test_loss, test_accuracy):
    predictions = model(x)
    loss = loss_op(y, predictions)

    test_loss(loss)
    test_accuracy(y, predictions)

In [4]:
def getData():
    x_train, y_train, x_test, y_test = utils.getCifar10Dataset()
    train_data, x_test, y_test = dataProcessing.preprocessData(x_train, y_train, x_test, y_test, batch_size=128)
    return train_data, x_test, y_test

train_data, x_test, y_test = getData()

print('Data shape: {}'.format(x_test.shape[1:]))

Data shape: (32, 32, 3)


In [15]:
def train(model, train_data, x_test, y_test, loss_op, optimization, epochs):
    train_loss = tf.keras.metrics.Mean(name='train_loss')
    train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')
    test_loss = tf.keras.metrics.Mean(name='test_loss')
    test_accuracy = tf.keras.metrics.CategoricalAccuracy(name='test_accuracy')

    n_batches = len(train_data)

    for epoch in range(epochs):
        n_batch = 0
        for x, y in train_data:
            n_batch+=1
            template = '[Batch {}/{}] Loss: {:.3f}, Accuracy: {:.2%}'
            print(template.format(n_batch, n_batches, train_loss.result(), train_accuracy.result()), end='\r')
            trainStep(model, x, y, loss_op, optimization, train_loss, train_accuracy)
            
            if n_batch >= n_batches:
                # we need to break the loop by hand because
                # the generator loops indefinitely
                break

        testStep(model, x_test, y_test, loss_op, test_loss, test_accuracy)

        template = '\n[Epoch {}/{}] Loss: {:.3f}, Accuracy: {:.2%}, Test Loss: {:.3f}, Test Accuracy: {:.2%}'
        print(template.format(epoch+1, epochs, train_loss.result(), train_accuracy.result(),
                test_loss.result(), test_accuracy.result()))

        # Reset the metrics for the next epoch
        train_loss.reset_states()
        train_accuracy.reset_states()
        test_loss.reset_states()
        test_accuracy.reset_states()

    return model

In [16]:
learning_rate = 0.1
epochs = 5

loss_op = keras.losses.CategoricalCrossentropy()
optimizer = keras.optimizers.Adam(lr=learning_rate)

In [17]:
# First train a normal 2D conv network
model = keras.models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='relu'))

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_25 (Conv2D)           (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 4, 4, 64)          36928     
_________________________________________________________________
flatten_4 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 64)               

In [18]:
train(model, train_data, x_test, y_test, loss_op, optimizer, epochs)

[Batch 256/391] Loss: 6.239, Accuracy: 10.92%